# Run Model
Dit notebook kan gebruikt worden om BrfRFE_model1 te draaien. Dit model is bedoeld om te voorspellen of een kandidaat starter zal worden op basis van de data die verzamelt is voordat het eerste telefonisch contact is geweest. 

In [1]:
# !git clone https://github.com/peterfaasse/BastianLewisModelRepo.git

## Import necessary libraries

In [2]:
import pandas as pd
import numpy as np
import os
import json
from datetime import datetime
import re
from helper import get_most_recent_file, make_dir
from data_prep import run_data_prep, prepare_input_dataframe, prep_dataset_for_modelling, replace_invalid_values

## Upload data
Je kan de data van de kandidaten waarover je wilt voorspellen of ze starter worden uploaden in de map "kandidaat_data". Dit dient in de vorm van een excel te zijn met de naam "kandidaten_[DATUM].xlsx". Op de plek van [DATUM] vul je de datum in van uploaden in het format yyyymmdd, zonder streepjes of iets er tussen. Bijvoorbeeld een data bestand geupload op 11 augustus 2024 heeft de naam "kandidaten_20240811.xlsx". 

De excel heeft een aantal vaste kolommen en kolomnamen. Een voorbeeld excel bevind zich in kandidaten_data/voorbeeld_kandidaten.xlsx. Let op dat wijzigingen in de namen van kolommen of antwoordcategorieen impact hebben op de bruikbaarheid van de data en de kwaliteit van het model. 

## Set directories and load data

In [3]:
kandidaten_dir = r"C:\Users\peter\Projects\HR_Forecasting\kandidaat_data"
data_dir = r"C:\Users\peter\Projects\HR_Forecasting\data"
model_dir = r"C:\Users\peter\Projects\HR_Forecasting\models"
supporting_data_dir = r"C:\Users\peter\Projects\HR_Forecasting\supporting_data"
file_path = get_most_recent_file(kandidaten_dir)
df_kandidaten = pd.read_excel(file_path)

## Lees model en metadata in

In [4]:
## select model
model_name = "model_3" # or select "model_2"

## set paths
model_path = os.path.join(model_dir, model_name + ".pkl")
meta_path = os.path.join(model_dir, model_name+"_meta.json")

In [5]:
#laad model en metadata
brf_rfe = pd.read_pickle(model_path)
with open(meta_path, 'r') as file:
    model_meta = json.load(file)

## Prepareer data voor forecasting

In [6]:
#laad dataset metadata in voor dit model
json_path = os.path.join(model_dir,f"{model_meta['dataset'].split('_final')[0]}_categorical_meta_data.json")
with open(json_path, "r") as file:
    categorical_meta_data = json.load(file)

In [7]:
#prep dataframe, select only model variables fil missings
df_model_input = run_data_prep(df_kandidaten, supporting_data_dir)
df_model_input = replace_invalid_values(df_model_input, categorical_meta_data)
df_model_input = prepare_input_dataframe(df_model_input, model_meta['selected_features'])
df_model_input.fillna(999, inplace=True)

removed 0 candidates who did not have prioriteit 1
removed 0 candidates who did not have NULL for afwijsBasisGegevens


In [9]:
df_model_input

,cdate,recruitercode,belafspraak,leeftijd,geboortedatum,Verloning marge stopper of W&S overname,jaar_ervaring,diff_days,Voorkeursbranche_elektrotechniek,Voorkeursbranche_installatietechniek
0,1627451332000000000,999,-9223372036854775808,999.0,-9223372036854775808,1031042.0,999.0,999.0,999,999
1,1627463776000000000,999,-9223372036854775808,999.0,-9223372036854775808,999.0,999.0,999.0,999,999
2,1627645583000000000,999,1668698100000000000,21.0,-9223372036854775808,999.0,999.0,475.0,999,999


## Make predictions and save in Results

In [10]:
predictions = brf_rfe.predict(df_model_input)
probas = brf_rfe.predict_proba(df_model_input)
model_pred = pd.DataFrame()
model_pred['id'] = list(df_kandidaten["id"])
model_pred['pred'] = list(predictions)
model_pred['prob'] = probas[:,1]

datetimestring = re.sub(r'\D', '', str(datetime.today()))
make_dir("results")
output_path = f"results/{model_name}_predictions_{datetimestring}.xlsx"
model_pred.to_excel(output_path, index=False)

c:\Users\peter\anaconda3\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but BalancedRandomForestClassifier was fitted without feature names
  warnings.warn(
c:\Users\peter\anaconda3\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but BalancedRandomForestClassifier was fitted without feature names
  warnings.warn(


Directory already exists.


In [11]:
model_pred

,id,pred,prob
0,244,1,0.82
1,245,1,0.66
2,261,1,0.62


## Todos
1. Fix nieuwe kolomwaardes naar other
2. Vergelijk model oude data met model nieuwe data
3. Voeg schone textdata toe aan data en draai modellen opnieuw
4. Voeg map toe met model evaluatie metrics en trees
5. Creeer aparte environment